In [ ]:
# Importing the necessary packages 
import mlflow 
from mlflow.models.signature import infer_signature
from ultralytics import YOLO 
import cv2 
from PIL import Image
import requests
from io import BytesIO
import io 
import base64

import matplotlib.pyplot as plt

# Defining the path to the model 
model_path = "runs/segment/train14/weights/last.pt"

# Initiating the model
model = YOLO(model_path)

In [ ]:
# Defining the link to an image on the web 
#image_link = "https://cdn-gihjl.nitrocdn.com/IWCueMOBANIXxJmBaspSaptvIdnHUnAd/assets/images/optimized/rev-9a82a2b/www.robertlowdon.com/wp-content/uploads/2019/08/agriculture-photography-004.jpg"
image_link = 'ml_data/images/val/UI_2024-09-30_054907_UTC_SosnovskioPapildymas_rugsejis_DJI_0106_W.JPG'

# Loading the image from the web
if image_link.startswith("http"):
    response = requests.get(image_link)
    image = Image.open(BytesIO(response.content))
else:
    image = Image.open(image_link)

# Ploting the image 
plt.imshow(image)

In [ ]:
def pil_image_to_base64(image, image_format="PNG"):
    # Create an in-memory bytes buffer
    buffered = io.BytesIO()
    # Save the PIL image to this buffer in the specified format
    image.save(buffered, format=image_format)
    
    # Get the raw bytes from the buffer
    img_bytes = buffered.getvalue()
    
    # Encode these bytes to base64
    img_base64 = base64.b64encode(img_bytes)
    
    # Convert to a UTF-8 string
    img_str = img_base64.decode('utf-8')
    return img_str

In [ ]:
# Converting to base64
image_base64 = pil_image_to_base64(image)

In [ ]:
print(image_base64)

In [ ]:
# Encoding the image to base64 format 
image.save("image.jpg")
with open("image.jpg", "rb") as f:
    image_bytes = f.read()
image_base64 = str(image_bytes)

In [ ]:
predictions = model.predict(image)

In [ ]:
boxes = predictions[0].boxes.xyxy.detach().cpu().numpy()
masks = predictions[0].masks
if masks is not None: 
    masks = masks.xy

In [ ]:
output = {
    "boxes": boxes,
    "masks": masks
}

In [ ]:
output

In [ ]:
model_signature = infer_signature(model_input=image_link, model_output=output)